# European Clima Law

In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import unicodedata

In [2]:
## Getting the law: Europäisches Klimagesetz

URL = "http://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:32021R1119&from=EN"
law = "Europäisches Klimagesetz"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 

c:\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
## Getting all the artikels and separating Artikels with and without Absatze:

#1. getting all artikels and absatze:

ids = [tag['id'] for tag in soup.select('div[id]')]

df = pd.DataFrame(ids)
df.rename(columns={list(df)[0]:'ids'}, inplace=True)
df['artikel'] = df['ids'].str[:3]
#df

,ids,artikel
0,001,001
1,002,002
2,002.001,002
3,002.002,002
4,003,003
5,003.001,003
6,003.002,003
7,003.003,003
8,003.004,003
9,004,004


In [40]:
#1. Creating empty df:
df_law = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])

#2. loop for ids:

for id_entry in ids:
    #pegando tudo:
    table = soup.find('div', attrs = {'id':id_entry}) 
    my_str = table.text
    result = unicodedata.normalize('NFKD', my_str)   # removind \xa0
    result = " ".join(result.split())
    
    if id_entry.count('.') == 0:                                               # Se nao tem absatz
        df_temp = pd.DataFrame({'original_id' : [id_entry],
                                'gesetzt': [law],
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : ["NA"],
                                'text': [result]})
    else:
        df_temp = pd.DataFrame({'original_id' : [id_entry],
                                'gesetzt': [law],
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : [id_entry.split('.')[1]],
                                'text': [result]})
    #df_temp.dtypes funciona
    df_law = pd.concat([df_law, df_temp])

# Vou dropar os ids problemáticos, porque como eles sao de artigos sem absatz, eu posso só deixar o absatz

ids_corr = ['010A','045', '010A.001', '010A.002', '010A.003', '010A.004', '010A.005', '011.005', '015.001']

for id in ids_corr:
    df_law = df_law[df_law.original_id != id]

#o item 011 tßa duplicado, eu tentei retirar de tudo quanto é forma mas nao, deu, vou tirar na mao

df_law.reset_index(inplace = True)

df_law.drop([44], axis=0, inplace=True)
len(df_law)

45

In [48]:
#Criando um conjunto de artigos com absatz

df_mit_ab = df_law.groupby(by = 'artikel').agg(absatz = ("artikel", "count"))
df_mit_ab = df_mit_ab.query("absatz > 1")
df_mit_ab.reset_index(inplace=True)


art_mit_ab = df_mit_ab['artikel'].tolist()
art_mit_ab


['002', '003', '004', '005', '006', '007', '008', '009']

In [56]:
#Removendo a primeira entrada dos artigos com absatz do meu df, para nao ter problema:

for art in art_mit_ab:
    df_law = df_law[df_law.original_id != art]

In [57]:
#Validando

len(df_law)  # 37 e conferi item e item, está certo. 

37

In [58]:
#Salvando o banco com esse artigo:

df_law.to_csv("eu_klimagesetz.csv")